In [134]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=1505e733ae385026c13f838a71b56d864d88790477915153d1c18123474f32a0
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 21.6 GB  | Proc size: 16.1 GB
GPU RAM Free: 407MB | Used: 11034MB | Util  96% | Total 11441MB


In [0]:
!kill -9 -1

In [1]:
# importing the libraries
import pandas as pd
import numpy as np

import os
from keras import backend as K
from keras.layers.normalization import BatchNormalization

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


from keras.callbacks import ModelCheckpoint
import numpy
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
cwd_train = "/content/gdrive/My Drive/comp551/data/train_max_x"
cwd_test = "/content/gdrive/My Drive/comp551/data/test_max_x"
cwd_targets = "/content/gdrive/My Drive/comp551/data/"

In [0]:
train_dataset = pd.read_pickle(cwd_train) 


targets = pd.read_csv(os.path.join(cwd_targets, 'train_max_y.csv'))
targets= targets.iloc[:,1]


#X_train = train_dataset
#y_train = targets

X_train, X_test, y_train, y_test = train_test_split(train_dataset, targets, test_size=0.2, random_state=42)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_dataset, targets, test_size=0.2)

In [0]:
test_dataset = pd.read_pickle(cwd_test)
X_test_set = test_dataset

In [0]:
X_test_set = X_test_set.astype('float32')
X_test_set /= 255

In [0]:
if K.image_data_format() == 'channels_first':
  X_test_set = X_test_set.reshape(X_test_set.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
   X_test_set = X_test_set.reshape(X_test_set.shape[0], img_rows, img_cols, 1)
   input_shape = (img_rows, img_cols, 1)

In [0]:
img_rows=128
img_cols=128

In [8]:
#reshaping
#this assumes our data format
#For 3D data, "channels_last" assumes (conv_dim1, conv_dim2, conv_dim3, channels) while 
#"channels_first" assumes (channels, conv_dim1, conv_dim2, conv_dim3).
if K.image_data_format() == 'channels_first':
  X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
  X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
   X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
   X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
   input_shape = (img_rows, img_cols, 1)
#more reshaping

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape) 

X_train shape: (40000, 128, 128, 1)


In [25]:
X_train.shape

(40000, 128, 128, 1)

In [26]:
y_train.shape

(40000, 10)

In [0]:
import keras
#set number of categories
num_category = 10

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_category)
y_test = keras.utils.to_categorical(y_test, num_category)

In [0]:
IMG_SIZE = 128
IMG_SIZE = 128

In [14]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(kernel_size=3, filters=16, padding='same', activation='relu', input_shape=input_shape),
#     tf.keras.layers.Conv2D(kernel_size=3, filters=30, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=2),
#     tf.keras.layers.Conv2D(kernel_size=3, filters=60, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=2),
#     tf.keras.layers.Conv2D(kernel_size=3, filters=90, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=2),
#     tf.keras.layers.Conv2D(kernel_size=3, filters=110, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=2),
#     tf.keras.layers.Conv2D(kernel_size=3, filters=130, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(kernel_size=1, filters=40, padding='same', activation='relu'),
#     tf.keras.layers.GlobalAveragePooling2D(),
    
#     tf.keras.layers.Dense(num_category, activation='softmax')
#     #tf.keras.layers.Dense(1,'sigmoid')
# ])




Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# model = Sequential()

# model.add(Conv2D(32, (3, 3), input_shape=input_shape))
# model.add(Activation('relu'))
# BatchNormalization(axis=-1)
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# BatchNormalization()
# model.add(Dense(10))
# model.add(Activation('softmax'))

In [0]:



# model = Sequential()

# model.add(Conv2D(32, (3, 3), input_shape=input_shape))
# model.add(Activation('relu'))
# BatchNormalization(axis=-1)
# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# BatchNormalization(axis=-1)
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# BatchNormalization(axis=-1)
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())

# BatchNormalization()
# model.add(Dense(512))
# model.add(Activation('relu'))
# BatchNormalization()
# model.add(Dropout(0.2))
# model.add(Dense(10))

# model.add(Activation('softmax'))

#####################################




model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))

BatchNormalization(axis=-1)
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))

model.add(Flatten())

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))




In [0]:
model.compile(optimizer='Adam',
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [0]:
epochs=40
batch_size = 256

# NOTES: 
# batch size: 
# 256 converges at epoch 9


In [53]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

Train on 40000 samples, validate on 10000 samples
Epoch 1/40
40000/40000 [==============================] - 59s 1ms/step - loss: 1.9041 - acc: 0.2620 - val_loss: 1.8986 - val_acc: 0.2613
Epoch 2/40
40000/40000 [==============================] - 58s 1ms/step - loss: 1.8869 - acc: 0.2682 - val_loss: 1.8858 - val_acc: 0.2613
Epoch 3/40
40000/40000 [==============================] - 58s 1ms/step - loss: 1.8797 - acc: 0.2691 - val_loss: 2.3127 - val_acc: 0.2613
Epoch 4/40
40000/40000 [==============================] - 58s 1ms/step - loss: 1.7646 - acc: 0.3178 - val_loss: 1.4821 - val_acc: 0.4466
Epoch 5/40
40000/40000 [==============================] - 58s 1ms/step - loss: 1.1596 - acc: 0.5913 - val_loss: 0.9430 - val_acc: 0.6782
Epoch 6/40
40000/40000 [==============================] - 58s 1ms/step - loss: 0.7452 - acc: 0.7511 - val_loss: 0.7018 - val_acc: 0.7697
Epoch 7/40
40000/40000 [==============================] - 58s 1ms/step - loss: 0.5442 - acc: 0.8179 - val_loss: 0.5891 - val_acc

KeyboardInterrupt: ignored

In [54]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]) 


Test loss: 0.5967727069377899
Test accuracy: 0.835


In [0]:
ynew = model.predict_classes(X_test_set)

In [0]:
Id = list(range(0, 10000))

In [0]:
Id = np.asarray(Id)

In [0]:
percent85 = pd.DataFrame({'Id': Id, 'Label': ynew}, columns=['Id', 'Label'])

In [0]:
# dataset = pd.DataFrame({'Id': Id, 'Label': ynew}, columns=['Id', 'Label'])


In [0]:
# from google.colab import files
# dataset.to_csv('85%_2Nov.csv', index=False) 

In [0]:
from google.colab import files
percent85.to_csv('83.5_2Nov.csv', index=False)